In [1]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import time
warnings.simplefilter(action="ignore", category=FutureWarning)
import os, sys
sys.path.append('../')
from src.utils.params import hash_dict
from src.utils.serialize import pickle_obj, unpickle_obj, load_json, save_json

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def logistic(x):
    return 1/(1+np.exp(-x))

In [ ]:
output_dir = '.'
num_chains = 1
nsim = 500

for d in [10, 25, 50, 100]:
    for T in [10, 25, 50, 75, 100, 150]:
        for method in ['nuts']:

            params = {
                'd' : d,
                'T' : T,
                'seed': 123,
                'method': method
            }
            print(params)
            seed = params['seed']

            # run masked bps
            model_output_dir = os.path.join(output_dir)

            if not os.path.exists(model_output_dir):
                    os.mkdir(model_output_dir)

            ## set up output dir
            param_hash = hash_dict(params)
            dir_name = "experiment_{0}".format(param_hash)
            dir_path = os.path.join(model_output_dir, dir_name)
            if not os.path.exists(dir_path):
                os.mkdir(dir_path)
            
            mu0 = np.array([0. for _ in range(d)])
            sig0 = np.diag([1. for _ in range(d)])
            
            if 'results.pkl' not in os.listdir(dir_path):
                nodes = {}
                with pm.Model() as first_model:
                    #priors on parameters
                    nodes[0] = pm.MvNormal('beta', mu=mu0, cov=sig0, shape=(d,))
                    for t in range(1,T+1):
                        nodes[t] = pm.MvNormal('b_{0}'.format(t), mu=nodes[t-1], cov=sig0,shape=(d,))
                        
                start = time.time()
                with first_model:    
                    #samples from posterior distribution 
                    trace=pm.sample(nsim, tune=500, cores=num_chains, chains=num_chains, compute_convergence_checks=False)
                duration = time.time()-start
                
                df = az.summary(trace, round_to=5)
                ess_speed = np.mean(df['ess_bulk']/duration/num_chains) * (nsim+500)/nsim
                results = {
                    "ess_speed" : ess_speed.item(),
                    'T' : T,
                    'd' : d
                }
                pickle_obj(results, os.path.join(dir_path, 'results.pkl'))


{'d': 10, 'T': 10, 'seed': 123, 'method': 'nuts'}
{'d': 10, 'T': 25, 'seed': 123, 'method': 'nuts'}
{'d': 10, 'T': 50, 'seed': 123, 'method': 'nuts'}


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Sequential sampling (1 chains in 1 job)
NUTS: [b_50, b_49, b_48, b_47, b_46, b_45, b_44, b_43, b_42, b_41, b_40, b_39, b_38, b_37, b_36, b_35, b_34, b_33, b_32, b_31, b_30, b_29, b_28, b_27, b_26, b_25, b_24, b_23, b_22, b_21, b_20, b_19, b_18, b_17, b_16, b_15, b_14, b_13, b_12, b_11, b_10, b_9, b_8, b_7, b_6, b_5, b_4, b_3, b_2, b_1, beta]


In [ ]:
pickle_obj(results, os.path.join(dir_path, 'results.pkl'))

In [ ]:
results